<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Auto-Complete-Beginner/blob/main/NLP_Auto_Complete_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!pip install -q -U trax

import trax

# import trax.fastmath.numpy as np

from trax import layers as tl

     |████████████████████████████████| 637 kB 8.2 MB/s 
     |████████████████████████████████| 4.9 MB 56.2 MB/s 
     |████████████████████████████████| 462 kB 53.9 MB/s 


In [3]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%203/en_US.twitter.txt

--2022-04-25 17:27:30--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%203/en_US.twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341555 (3.2M) [text/plain]
Saving to: ‘en_US.twitter.txt’

en_US.twitter.txt   100%[===================>]   3.19M  --.-KB/s    in 0.05s   

2022-04-25 17:27:30 (60.8 MB/s) - ‘en_US.twitter.txt’ saved [3341555/3341555]



In [4]:
with open('en_US.twitter.txt', 'r') as f:
  data = f.read()

In [5]:
print(len(data), type(data))

3335477 <class 'str'>


In [6]:
display(data[0:400])

"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)\nWords from a complete stranger! Made my birthday eve"

In [7]:
display(data[-400:])

"tplace jerks off aristocrats until they finally die in the midst of self-love.\n's icon is life.\nwe just had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

In [8]:
def split2Sentences(data):
  
  sentences = data.split('\n')
  sentences = [s.strip() for s in sentences]
  sentences = [s for s in sentences if len(s)>0]
  return sentences

In [9]:
sentences = split2Sentences(data)
sentences[0:10]

['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.',
 "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.",
 "they've decided its more fun if I don't.",
 'So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)',
 'Words from a complete stranger! Made my birthday even better :)',
 'First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!',
 'i no! i get another day off from skool due to the wonderful snow (: and THIS wakes me up...damn thing',
 "I'm coo... Jus at work hella tired r u ever in cali",
 'The new sundrop commercial ...hehe love at first sight',
 'we need to reconnect THIS WEEK']

In [10]:
nltk.word_tokenize(sentences[2])

['they', "'ve", 'decided', 'its', 'more', 'fun', 'if', 'I', 'do', "n't", '.']

In [13]:
nltk.word_tokenize(sentences[20])

['“',
 ':',
 '``',
 'The',
 'tragedy',
 'of',
 'life',
 'is',
 'not',
 'that',
 'it',
 'ends',
 'so',
 'soon',
 ',',
 'but',
 'that',
 'we',
 'wait',
 'so',
 'long',
 'to',
 'begin',
 'it',
 '.',
 "''",
 '-',
 'W.M',
 '.',
 'Lewis',
 '”']

In [14]:
tokenize_sentences =lambda sentences: [nltk.word_tokenize(s.lower()) for s in sentences]

In [15]:
tokenized_sentences = tokenize_sentences(sentences)

In [16]:
tokenized_sentences[0]

['how',
 'are',
 'you',
 '?',
 'btw',
 'thanks',
 'for',
 'the',
 'rt',
 '.',
 'you',
 'gon',
 'na',
 'be',
 'in',
 'dc',
 'anytime',
 'soon',
 '?',
 'love',
 'to',
 'see',
 'you',
 '.',
 'been',
 'way',
 ',',
 'way',
 'too',
 'long',
 '.']

In [17]:
tokenized_sentences[20]

['“',
 ':',
 '``',
 'the',
 'tragedy',
 'of',
 'life',
 'is',
 'not',
 'that',
 'it',
 'ends',
 'so',
 'soon',
 ',',
 'but',
 'that',
 'we',
 'wait',
 'so',
 'long',
 'to',
 'begin',
 'it',
 '.',
 "''",
 '-',
 'w.m',
 '.',
 'lewis',
 '”']

In [ ]:
data_tokenizer = lambda data: tokenize_sentences(split2Sentences(data))

In [ ]:
tokenized_sentences = data_tokenizer(data)

In [ ]:
np.random.shuffle(tokenized_sentences)
train_size = int(len(tokenized_sentences)*0.8)
train_data, test_data = tokenized_sentences[:train_size], tokenized_sentences[train_size:]

In [ ]:
print(len(train_data), len(test_data))

38368 9593


In [ ]:
def word_counter(tokenized_sentences):
  word_dic = {}
  for sentence in tokenized_sentences:
    for token in sentence:
      if token not in word_dic:
        word_dic[token] = 1
      else:
        word_dic[token] += 1
  return word_dic
  # return collections.Counter(tokenized_sentences)

In [ ]:
word_c = word_counter(tokenized_sentences)

for i, w in enumerate(word_c):
  if i == 20:
    break

  print(w)

crate
and
barrel
?
girl
,
i
've
been
on
etsy
since
yesterday
.
this
stupid
website
is
spammin
all


In [ ]:
def words_with_nplus_frequncy(tokenize_sentences, count_threshold):
  closed_vocab = []
  word_count = word_counter(tokenized_sentences)
  for k, v in word_count.items():
    if v >= count_threshold:
      closed_vocab.append(k)
  return closed_vocab

In [ ]:
closed_vocab = words_with_nplus_frequncy(tokenized_sentences,2)

In [ ]:
closed_vocab[: 20]

['crate',
 'and',
 'barrel',
 '?',
 'girl',
 ',',
 'i',
 "'ve",
 'been',
 'on',
 'etsy',
 'since',
 'yesterday',
 '.',
 'this',
 'stupid',
 'website',
 'is',
 'all',
 'of']

In [ ]:
def oov2Unk(tokenized_data, vocab, unknown_token='<unk>'):
  vocab = set(vocab)

  replaced_tokenized_sentences = []
  for i, sentence in enumerate(tokenized_data):

    replace_sentences = []
    for token in sentence:

      temp_token = token if token in vocab else unknown_token
      replace_sentences.append(temp_token)
    replaced_tokenized_sentences.append(replace_sentences)
  return replaced_tokenized_sentences

In [ ]:
def preprocess_data(train_data, test_data, count_threshol=2):
  
  vocab = words_with_nplus_frequncy(train_data, count_threshol)
  train_data_replaced = oov2Unk(train_data, vocab)
  test_data_replaced = oov2Unk(test_data, vocab)
  return train_data_replaced, test_data_replaced, vocab

In [ ]:
train_data_replaced, test_data_replaced, vocab = preprocess_data(train_data, test_data)

In [ ]:
train_data_replaced[20]

['we',
 'offer',
 'a',
 'free',
 'market',
 'approach',
 'to',
 'success',
 'in',
 'the',
 'music',
 'industry',
 '!',
 '!']

In [ ]:
test_data_replaced[20]

['wow',
 'i',
 'really',
 'wish',
 'i',
 'was',
 'back',
 'in',
 'high',
 'school',
 'right',
 'now',
 '...',
 '..']

In [ ]:
def creat_n_grams(data, n, start_token='<s>', end_token='<e>'):
  n_grams = {}
  for sentence in data:
    sentence = [start_token]*n + sentence + [end_token]
    sentence = tuple(sentence)
    m = len(sentence) if n==1 else len(sentence)-1

    for i in range(m):
      n_gram = sentence[i: i+n]
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      else:
        n_grams[n_gram] = 1
  return n_grams

In [ ]:
for i in creat_n_grams(test_data_replaced, 1).items():
  print(i)
  break

(('<s>',), 9593)


In [ ]:
creat_n_grams(test_data_replaced, 1)

{('<s>',): 9593,
 ('yo',): 26,
 ('i',): 3647,
 ("'m",): 526,
 ('serious',): 12,
 ('following',): 51,
 ('the',): 3877,
 ('next',): 133,
 ('10.',): 2,
 ('as',): 276,
 ('long',): 74,
 ('u',): 322,
 ('ai',): 19,
 ("n't",): 855,
 ('that',): 1129,
 ('<unk>',): 5106,
 ('shit',): 69,
 ('...',): 1090,
 ('.',): 5936,
 ('let',): 170,
 ("'s",): 1233,
 ('be',): 795,
 ('real',): 61,
 ('bro..',): 1,
 ('<e>',): 9593,
 ('lobster',): 2,
 ('&',): 419,
 ('shrimp',): 3,
 ('with',): 698,
 ('a',): 2440,
 ('suit',): 4,
 ('on',): 1091,
 ('today',): 305,
 ('is',): 1567,
 ('feast',): 4,
 ('day',): 378,
 ('of',): 1450,
 ('st',): 6,
 ('mark',): 6,
 ('evangelist',): 1,
 ('will',): 403,
 ('visiting',): 7,
 ('his',): 130,
 ('church',): 13,
 ('in',): 1548,
 ('venice',): 2,
 ('less',): 25,
 ('than',): 115,
 ('3',): 189,
 ('weeks',): 31,
 ('cheering',): 3,
 ('up',): 461,
 ('its',): 141,
 ('so',): 688,
 ('hard',): 67,
 ('to',): 3101,
 ('get',): 482,
 ('out',): 468,
 ('my',): 1173,
 ('bed',): 41,
 ('what',): 504,
 ('was',

In [ ]:
for i in creat_n_grams(test_data_replaced, 2).items():
  print(i)
  break

(('<s>', '<s>'), 9593)


In [ ]:
def probability_calculator(word, previous_n_gram,n_gram_counts,n_plus1_gram_counts,vocab_size, k=1.0):

  previous_n_gram = tuple(previous_n_gram)
  previous_n_gram_counts = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  denominator = previous_n_gram_counts + k*vocab_size
  n_plus1_gram = previous_n_gram + (word,)
  n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts  else 0
  numerator = n_plus1_gram_count + k
  probability = numerator / denominator
  return probability    

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = creat_n_grams(sentences, 1)
bigram_counts = creat_n_grams(sentences, 2)
tmp_prob = probability_calculator("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

In [ ]:
tmp_prob

0.3333333333333333

In [ ]:
def estimate_probabilities(previous_n_gram,n_gram_counts, n_plus1_gram_counts, vocab, k=1.0):

  previous_n_gram = tuple(previous_n_gram)
  vocab = vocab+['<e>', '<unk>']
  vocab_size = len(vocab)

  probabilities = {}
  for word in vocab:
    prob = probability_calculator(word, previous_n_gram,n_gram_counts, n_plus1_gram_counts, vocab_size, k=1.0)
    probabilities[word] = prob
  return probabilities

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = creat_n_grams(sentences, 1)
bigram_counts = creat_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091,
 'a': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'dog': 0.09090909090909091,
 'i': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'this': 0.09090909090909091}

In [ ]:
def count_matrix(n_plus1_gram_counts, vocab):
  vocab = vocab + ['<e>', '<unk>']
  n_grams = []
  for n_plus1_gram in n_plus1_gram_counts.keys():
    n_gram = n_plus1_gram[0: -1]
    n_grams.append(n_gram)
  n_grams = list(set(n_grams))

  row_index = {n_gram: i for i, n_gram in enumerate(n_grams)}
  col_index = {n_gram: j for j, n_gram in enumerate(vocab)}
  row = len(n_grams)
  col = len(vocab)
  count_mat = np.zeros((row, col))

  for n_plus1_gram, count in n_plus1_gram_counts.items():
    n_gram  = n_plus1_gram[0: -1]
    word = n_plus1_gram[-1]
    if word not in vocab:
      continue
    i, j = row_index[n_gram], col_index[word]
    count_mat[i, j] = count

  count_dic = pd.DataFrame(count_mat, index=n_grams, columns=vocab)
  return count_dic

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = creat_n_grams(sentences, 2)

print('unigram counts')
display(count_matrix(bigram_counts, unique_words))

unigram counts


,like,cat,this,dog,i,is,a,<e>,<unk>
"(<s>,)",0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
"(is,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like,)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(dog,)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(this,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(i,)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def probability_matrix(n_plus1_gram_counts, vocab, k):
  count_mat = count_matrix(n_plus1_gram_counts, vocab)
  count_mat += k
  prob_mat = count_mat.div(np.sum(count_mat, axis=1), axis=0)
  return prob_mat

In [ ]:
probability_matrix(bigram_counts, unique_words, k=1)

,like,cat,this,dog,i,is,a,<e>,<unk>
"(<s>,)",0.090909,0.090909,0.181818,0.090909,0.181818,0.090909,0.090909,0.090909,0.090909
"(is,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(like,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909
"(dog,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(this,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(i,)",0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [ ]:
def perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocab_size, k=1):

  n = len(list(n_gram_counts.keys())[0])
  sentence = ['<s>'] * n + sentence + ['<e>']
  sentence = tuple(sentence)
  N = len(sentence)
  product_pi = 1.0

  for i in range(n, N):
    n_gram = sentence[i-n: i]
    word = sentence[i]
    probability = probability_calculator(word, n_gram, n_gram_counts, n_plus1_gram_counts,vocab_size, k)
    product_pi /= probability
  
  perplexity_value = product_pi ** (1/N)
  return perplexity_value

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]

perplexity_value = perplexity(sentences[0], unigram_counts, bigram_counts, len(unique_words), k=1.0)

In [ ]:
perplexity_value

2.8039657955522013

In [ ]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    
    n = len(list(n_gram_counts.keys())[0]) 
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram,n_gram_counts, n_plus1_gram_counts,vocabulary, k=k)
    suggestion = None
    max_prob = 0
    
    for word, prob in probabilities.items():
        if start_with != None and not word.startswith(start_with):
            continue
        if prob > max_prob:
            suggestion = word
            max_prob = prob
    return suggestion, max_prob

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


In [ ]:
def get_suggestions(previous_tokens, n, sentences, k=1.0, start_with=None):

    vocabulary = list(set(sentences[0] + sentences[1]))
    n_gram_counts_list = []

    for i in range(1, n):
      n_model_counts = creat_n_grams(sentences, i)
      n_gram_counts_list.append(n_model_counts)

    model_counts = len(n_gram_counts_list)
    suggestions = []

    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,n_plus1_gram_counts, vocabulary, k, start_with)
        suggestions.append(suggestion)
        
    return suggestions

In [ ]:
n = 5

sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]

previous_tokens = ["i", "like"]
tmp_suggest = get_suggestions(previous_tokens, n, sentences, k=1.0)

print(f"The previous words are '{' '.join(previous_tokens)}', the suggestions are:")
display(tmp_suggest)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727), ('a', 0.2), ('like', 0.1111111111111111)]

In [ ]:
previous_tokens = ["i", "am", 'to']
tmp_suggest = get_suggestions(previous_tokens, n, sentences, k=1.0)

print(f"The previous words are '{' '.join(previous_tokens)}', the suggestions are:")
display(tmp_suggest)

The previous words are 'i am to', the suggestions are:


[('like', 0.1111111111111111),
 ('like', 0.1111111111111111),
 ('like', 0.1111111111111111)]

In [ ]:
n = 3
previous_tokens = ["hey", "how", "are", "you"]

tmp_suggest = get_suggestions(previous_tokens, n, sentences, k=1.0)

print(f"The previous words are '{' '.join(previous_tokens)}', the suggestions are:")
display(tmp_suggest)

The previous words are 'hey how are you', the suggestions are:


[('like', 0.1111111111111111)]